In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

## [Parameters]

In [2]:
lr = 1e-3
batch_size = 64
epochs = 20

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## [Datasets]

In [5]:
# Dataset은 MNIST Dataset으로 32x32x1 이미지입니다.

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

trainset = torchvision.datasets.MNIST(root='./datasets',
                                      transform=transform, train=True, download=True) # 60000장
testset = torchvision.datasets.MNIST(root='./datasets',
                                     transform=transform, train=False, download=True) # 10000장

In [6]:
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)

## [Models]

In [7]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.C1 = nn.Conv2d(1, 6, kernel_size=5)
        self.S2 = nn.AvgPool2d(2)
        self.C3 = nn.Conv2d(6, 16, kernel_size=5)
        self.S4 = nn.AvgPool2d(2)
        self.C5 = nn.Conv2d(16, 120, kernel_size=5)
        self.F6 = nn.Linear(120, 84)
        self.Out = nn.Linear(84, 10)
            
    def forward(self, x):
        x = self.C1(x)
        x = F.sigmoid(self.S2(x))
        x = self.C3(x)
        x = self.S4(x)
        x = self.C5(x)
        x = self.F6(torch.squeeze(x))
        x = self.Out(x)
        
        return x
    
    
model = LeNet().to(device)
model

LeNet(
  (C1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (S2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (C3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (S4): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (C5): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (F6): Linear(in_features=120, out_features=84, bias=True)
  (Out): Linear(in_features=84, out_features=10, bias=True)
)

## [Train]

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [9]:
for epoch in range(epochs):    
    for images, labels in train_loader:       
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)              
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad() # gradient 값을 0으로 변경 
        loss.backward() # backpropagation, loss function의 gradient값을 .grad에 저장
        optimizer.step() # 계산된 gradient로 매개변수 업데이트
    
    print(f"Epoch [{epoch}/{epochs}] Loss: {loss.item():.4f}")

Epoch [0/20] Loss: 0.5629
Epoch [1/20] Loss: 0.3453
Epoch [2/20] Loss: 0.5558
Epoch [3/20] Loss: 0.2041
Epoch [4/20] Loss: 0.2162
Epoch [5/20] Loss: 0.0765
Epoch [6/20] Loss: 0.0385
Epoch [7/20] Loss: 0.0300
Epoch [8/20] Loss: 0.0125
Epoch [9/20] Loss: 0.0172
Epoch [10/20] Loss: 0.0239
Epoch [11/20] Loss: 0.0170
Epoch [12/20] Loss: 0.0055
Epoch [13/20] Loss: 0.0158
Epoch [14/20] Loss: 0.0030
Epoch [15/20] Loss: 0.0046
Epoch [16/20] Loss: 0.0201
Epoch [17/20] Loss: 0.0130
Epoch [18/20] Loss: 0.1180
Epoch [19/20] Loss: 0.0043


## [Result]

In [10]:
model.to('cpu')

for i in range(10):
    pred = torch.argmax(model(trainset[i][0])).item()
    label = trainset.targets[i]
    
    print(f"모델 예측값 : {pred}, 실제 값 : {label}")

모델 예측값 : 5, 실제 값 : 5
모델 예측값 : 0, 실제 값 : 0
모델 예측값 : 4, 실제 값 : 4
모델 예측값 : 1, 실제 값 : 1
모델 예측값 : 9, 실제 값 : 9
모델 예측값 : 2, 실제 값 : 2
모델 예측값 : 1, 실제 값 : 1
모델 예측값 : 3, 실제 값 : 3
모델 예측값 : 1, 실제 값 : 1
모델 예측값 : 4, 실제 값 : 4
